In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import math
import operator
import matplotlib as mpl
from sklearn.linear_model import LinearRegression

def calculateSeconds(timestr):
    ftr = [3600,60,1]
    seconds = sum([a*b for a,b in zip(ftr, map(int,timestr.split(':')))])
    return seconds

import datetime
def getWeekday(date):
    return datetime.datetime.strptime(date,"%d%b%Y").strftime("%A")

def stringRepresentation(input):
    daystring = ""
    if input < 10:
        daystring = "0" + str(input)
    else:
        daystring = str(input)
    return daystring

feestdagen = ["01JAN2020","13APR2020","01MAY2020","21MAY2020","01JUN2020","21JUL2020","15AUG2020","01NOV2020","11NOV2020","25DEC2020",
"01JAN2021","05APR2021","01MAY2021","13MAY2021","24MAY2021","21JUL2021","15AUG2021","01NOV2021","11NOV2021","25DEC2021",
"01JAN2022","18APR2022","01MAY2022","26MAY2022","06JUN2022","21JUL2022","15AUG2022","01NOV2022","11NOV2022","25DEC2022"]

In [2]:
Infrabel_data = pd.read_csv("Data_raw_punctuality_202007.csv")

for i in range(8,13):
    month = stringRepresentation(i)
    Infrabel_data2 = pd.read_csv("Data_raw_punctuality_2020" + stringRepresentation(i) + ".csv")
    frames = [Infrabel_data,Infrabel_data2]
    Infrabel_data = pd.concat(frames)

for i in range(1,13):
    month = stringRepresentation(i)
    Infrabel_data2 = pd.read_csv("Data_raw_punctuality_2021" + stringRepresentation(i) + ".csv")
    frames = [Infrabel_data,Infrabel_data2]
    Infrabel_data = pd.concat(frames)

for i in range(1,2):
    month = stringRepresentation(i)
    Infrabel_data2 = pd.read_csv("Data_raw_punctuality_2022" + stringRepresentation(i) + ".csv")
    frames = [Infrabel_data,Infrabel_data2]
    Infrabel_data = pd.concat(frames)

Look at departure trains:

In [3]:
df2 = Infrabel_data[Infrabel_data['RELATION_DIRECTION'].notna()]
df2 = df2[df2['PLANNED_TIME_DEP'].notna()]

df2 = df2[df2['RELATION_DIRECTION'].str.contains("ANTWERPEN-CENTRAAL")]
df2 = df2[df2['PTCAR_LG_NM_NL'].str.contains("ANTWERPEN-CENTRAAL")]
df2 = df2[df2['PLANNED_TIME_DEP'].str.contains("17:")]

#Look at trains trains between 17h06 and 17h07
df2 = df2[df2["PLANNED_TIME_DEP"].apply(calculateSeconds).between(61560,61620)]

df2["WEEKDAY"] = df2["DATDEP"].apply(getWeekday)
cols = df2.columns.tolist()
cols = cols[0:1] + cols[-1:] + cols[1:-1]
workdays = ["Monday","Tuesday","Wednesday","Thursday","Friday"]
df2 = df2[cols]
df2 = df2[df2["WEEKDAY"].isin(workdays)]
df2 = df2[~df2["DATDEP"].isin(feestdagen)]

df2.head()

,DATDEP,WEEKDAY,TRAIN_NO,RELATION,TRAIN_SERV,PTCAR_NO,LINE_NO_DEP,REAL_TIME_ARR,REAL_TIME_DEP,PLANNED_TIME_ARR,PLANNED_TIME_DEP,DELAY_ARR,DELAY_DEP,RELATION_DIRECTION,PTCAR_LG_NM_NL,LINE_NO_ARR,PLANNED_DATE_ARR,PLANNED_DATE_DEP,REAL_DATE_ARR,REAL_DATE_DEP
2962,01JUL2020,Wednesday,1838,IC 02,SNCB/NMBS,37,59/1,NaN,17:06:28,NaN,17:06:00,NaN,28.0,IC 02: ANTWERPEN-CENTRAAL -> OOSTENDE,ANTWERPEN-CENTRAAL,NaN,NaN,01JUL2020,NaN,01JUL2020
29567,01JUL2020,Wednesday,3138,IC 31,SNCB/NMBS,37,25,NaN,17:06:11,NaN,17:06:00,NaN,11.0,IC 31: ANTWERPEN-CENTRAAL -> BRUSSEL-ZUID,ANTWERPEN-CENTRAAL,NaN,NaN,01JUL2020,NaN,01JUL2020
106982,02JUL2020,Thursday,1838,IC 02,SNCB/NMBS,37,59/1,NaN,17:07:31,NaN,17:06:00,NaN,91.0,IC 02: ANTWERPEN-CENTRAAL -> OOSTENDE,ANTWERPEN-CENTRAAL,NaN,NaN,02JUL2020,NaN,02JUL2020
111616,02JUL2020,Thursday,3138,IC 31,SNCB/NMBS,37,25,NaN,17:08:39,NaN,17:06:00,NaN,159.0,IC 31: ANTWERPEN-CENTRAAL -> BRUSSEL-ZUID,ANTWERPEN-CENTRAAL,NaN,NaN,02JUL2020,NaN,02JUL2020
148365,03JUL2020,Friday,1838,IC 02,SNCB/NMBS,37,59/1,NaN,17:07:05,NaN,17:06:00,NaN,65.0,IC 02: ANTWERPEN-CENTRAAL -> OOSTENDE,ANTWERPEN-CENTRAAL,NaN,NaN,03JUL2020,NaN,03JUL2020


Choose train 1838

In [4]:
df2 = df2[df2["TRAIN_NO"] == 1838]
days = len(df2["DATDEP"].unique())
days_list = df2["DATDEP"].unique()
print(days_list)
print(days)
df2.head()

['01JUL2020' '02JUL2020' '03JUL2020' '06JUL2020' '07JUL2020' '08JUL2020'
 '09JUL2020' '10JUL2020' '13JUL2020' '14JUL2020' '15JUL2020' '17JUL2020'
 '22JUL2020' '16JUL2020' '23JUL2020' '20JUL2020' '24JUL2020' '27JUL2020'
 '29JUL2020' '28JUL2020' '30JUL2020' '31JUL2020' '03AUG2020' '04AUG2020'
 '05AUG2020' '06AUG2020' '07AUG2020' '10AUG2020' '11AUG2020' '13AUG2020'
 '14AUG2020' '17AUG2020' '18AUG2020' '19AUG2020' '20AUG2020' '21AUG2020'
 '24AUG2020' '25AUG2020' '26AUG2020' '27AUG2020' '28AUG2020' '31AUG2020'
 '01SEP2020' '02SEP2020' '03SEP2020' '04SEP2020' '07SEP2020' '08SEP2020'
 '09SEP2020' '10SEP2020' '11SEP2020' '14SEP2020' '15SEP2020' '16SEP2020'
 '17SEP2020' '18SEP2020' '21SEP2020' '22SEP2020' '23SEP2020' '24SEP2020'
 '25SEP2020' '28SEP2020' '30SEP2020' '01OCT2020' '02OCT2020' '05OCT2020'
 '06OCT2020' '07OCT2020' '08OCT2020' '12OCT2020' '13OCT2020' '14OCT2020'
 '15OCT2020' '16OCT2020' '19OCT2020' '20OCT2020' '22OCT2020' '23OCT2020'
 '26OCT2020' '27OCT2020' '29OCT2020' '28OCT2020' '3

,DATDEP,WEEKDAY,TRAIN_NO,RELATION,TRAIN_SERV,PTCAR_NO,LINE_NO_DEP,REAL_TIME_ARR,REAL_TIME_DEP,PLANNED_TIME_ARR,PLANNED_TIME_DEP,DELAY_ARR,DELAY_DEP,RELATION_DIRECTION,PTCAR_LG_NM_NL,LINE_NO_ARR,PLANNED_DATE_ARR,PLANNED_DATE_DEP,REAL_DATE_ARR,REAL_DATE_DEP
2962,01JUL2020,Wednesday,1838,IC 02,SNCB/NMBS,37,59/1,NaN,17:06:28,NaN,17:06:00,NaN,28.0,IC 02: ANTWERPEN-CENTRAAL -> OOSTENDE,ANTWERPEN-CENTRAAL,NaN,NaN,01JUL2020,NaN,01JUL2020
106982,02JUL2020,Thursday,1838,IC 02,SNCB/NMBS,37,59/1,NaN,17:07:31,NaN,17:06:00,NaN,91.0,IC 02: ANTWERPEN-CENTRAAL -> OOSTENDE,ANTWERPEN-CENTRAAL,NaN,NaN,02JUL2020,NaN,02JUL2020
148365,03JUL2020,Friday,1838,IC 02,SNCB/NMBS,37,59/1,NaN,17:07:05,NaN,17:06:00,NaN,65.0,IC 02: ANTWERPEN-CENTRAAL -> OOSTENDE,ANTWERPEN-CENTRAAL,NaN,NaN,03JUL2020,NaN,03JUL2020
294552,06JUL2020,Monday,1838,IC 02,SNCB/NMBS,37,59/1,NaN,17:06:24,NaN,17:06:00,NaN,24.0,IC 02: ANTWERPEN-CENTRAAL -> OOSTENDE,ANTWERPEN-CENTRAAL,NaN,NaN,06JUL2020,NaN,06JUL2020
393997,07JUL2020,Tuesday,1838,IC 02,SNCB/NMBS,37,59/1,NaN,17:07:22,NaN,17:06:00,NaN,82.0,IC 02: ANTWERPEN-CENTRAAL -> OOSTENDE,ANTWERPEN-CENTRAAL,NaN,NaN,07JUL2020,NaN,07JUL2020


Look at all ariving trains:

In [5]:
def getArrivalDataframe(station,depTrain,depTime,input_dataframe):
    output_dataframe = input_dataframe[input_dataframe['RELATION_DIRECTION'].notna()]
    output_dataframe = output_dataframe[output_dataframe['PLANNED_TIME_ARR'].notna()]
    output_dataframe = output_dataframe[output_dataframe['RELATION_DIRECTION'].str.contains(station)]
    output_dataframe = output_dataframe[output_dataframe['PTCAR_LG_NM_NL'].str.contains(station)]
    
    output_dataframe = output_dataframe[output_dataframe["PLANNED_TIME_ARR"].apply(calculateSeconds).between(calculateSeconds(depTime)-3600,calculateSeconds(depTime))]
    output_dataframe["WEEKDAY"] = output_dataframe["DATDEP"].apply(getWeekday)

    cols = output_dataframe.columns.tolist()
    cols = cols[0:1] + cols[-1:] + cols[1:-1]
    workdays = ["Monday","Tuesday","Wednesday","Thursday","Friday"]
    output_dataframe = output_dataframe[cols]
    output_dataframe = output_dataframe[output_dataframe["WEEKDAY"].isin(workdays)]
    output_dataframe = output_dataframe[~output_dataframe["DATDEP"].isin(feestdagen)]
    arriving_trains_list = output_dataframe["TRAIN_NO"].unique()

    train_frequency = output_dataframe["TRAIN_NO"].value_counts()
    train_frequency

    for train in arriving_trains_list:
        if (train_frequency[train] < days/5):
            output_dataframe = output_dataframe[output_dataframe["TRAIN_NO"] != train]
        
    arriving_trains_list = output_dataframe["TRAIN_NO"].unique()
    print(arriving_trains_list)
    print(len(arriving_trains_list))
    return output_dataframe,arriving_trains_list

In [6]:
arrivalDataframe, arriving_trains_list = getArrivalDataframe("ANTWERPEN-CENTRAAL",1838,"17:06:00",Infrabel_data)

[2636 1815 1764 2887  736 2936 2986 4215 1964 3315 4336 2686 3115 3036
 1487]
15


In [8]:
arrivalDataframe[arrivalDataframe["DATDEP"] == "03JAN2022"]

,DATDEP,WEEKDAY,TRAIN_NO,RELATION,TRAIN_SERV,PTCAR_NO,LINE_NO_DEP,REAL_TIME_ARR,REAL_TIME_DEP,PLANNED_TIME_ARR,PLANNED_TIME_DEP,DELAY_ARR,DELAY_DEP,RELATION_DIRECTION,PTCAR_LG_NM_NL,LINE_NO_ARR,PLANNED_DATE_ARR,PLANNED_DATE_DEP,REAL_DATE_ARR,REAL_DATE_DEP
93746,03JAN2022,Monday,1815,IC 02,SNCB/NMBS,37,NaN,16:53:37,NaN,16:54:00,NaN,-23.0,NaN,IC 02: OOSTENDE -> ANTWERPEN-CENTRAAL,ANTWERPEN-CENTRAAL,59/1,03JAN2022,NaN,03JAN2022,NaN
100785,03JAN2022,Monday,2936,IC 09,SNCB/NMBS,37,NaN,16:28:32,NaN,16:28:00,NaN,32.0,NaN,IC 09: LEUVEN -> ANTWERPEN-CENTRAAL,ANTWERPEN-CENTRAAL,27,03JAN2022,NaN,03JAN2022,NaN
105145,03JAN2022,Monday,4215,IC 30,SNCB/NMBS,37,NaN,16:13:45,NaN,16:13:00,NaN,45.0,NaN,IC 30: TURNHOUT -> ANTWERPEN-CENTRAAL,ANTWERPEN-CENTRAAL,59/1,03JAN2022,NaN,03JAN2022,NaN
117284,03JAN2022,Monday,736,IC 04-1,SNCB/NMBS,37,NaN,16:22:59,NaN,16:23:00,NaN,-1.0,NaN,IC 04-1: POPERINGE -> ANTWERPEN-CENTRAAL,ANTWERPEN-CENTRAAL,59/1,03JAN2022,NaN,03JAN2022,NaN
121919,03JAN2022,Monday,2636,IC 08,SNCB/NMBS,37,NaN,16:14:01,NaN,16:11:00,NaN,181.0,NaN,IC 08: HASSELT -> ANTWERPEN-CENTRAAL,ANTWERPEN-CENTRAAL,25,03JAN2022,NaN,03JAN2022,NaN
129092,03JAN2022,Monday,1764,L B1-1,SNCB/NMBS,37,NaN,16:23:39,NaN,16:22:00,NaN,99.0,NaN,L B1-1: NIVELLES -> ANTWERPEN-CENTRAAL,ANTWERPEN-CENTRAAL,27,03JAN2022,NaN,03JAN2022,NaN
129692,03JAN2022,Monday,2986,L A3,SNCB/NMBS,37,NaN,16:33:57,NaN,16:34:00,NaN,-3.0,NaN,L A3: MOL -> ANTWERPEN-CENTRAAL,ANTWERPEN-CENTRAAL,59/1,03JAN2022,NaN,03JAN2022,NaN
132779,03JAN2022,Monday,1964,L B1-2,SNCB/NMBS,37,NaN,16:55:31,NaN,16:55:00,NaN,31.0,NaN,L B1-2: NIVELLES -> ANTWERPEN-CENTRAAL,ANTWERPEN-CENTRAAL,27,03JAN2022,NaN,03JAN2022,NaN
133666,03JAN2022,Monday,2686,L A4-2,SNCB/NMBS,37,NaN,16:41:37,NaN,16:41:00,NaN,37.0,NaN,L A4-2: LOKEREN -> ANTWERPEN-CENTRAAL,ANTWERPEN-CENTRAAL,59/1,03JAN2022,NaN,03JAN2022,NaN
134758,03JAN2022,Monday,3036,IC 28,SNCB/NMBS,37,NaN,16:09:22,NaN,16:10:00,NaN,-38.0,NaN,IC 28: DE PANNE -> ANTWERPEN-CENTRAAL,ANTWERPEN-CENTRAAL,59/1,03JAN2022,NaN,03JAN2022,NaN


Some trains can exceptionally depart at a different hour. Delete the trains that are not running more than 50% of the days and thus are in the arriving train list by accident

In [9]:
def training_x_values(arrivaldf,arriving_tr,inputdays):
    training_xi = np.zeros((days, len(arriving_tr) + 1))
    training_xi[:,0] = 1 #set b 1
    for i in range(len(inputdays)):
        df_date = arrivaldf[arrivaldf["DATDEP"] == inputdays[i]]
        for train in df_date["TRAIN_NO"].values:
            train_index = df_date.loc[df_date['TRAIN_NO'] == train].index[0] #index in df
            trainlist_index = np.where(arriving_tr == train) #index in arriving_trains_list
            training_xi[i,trainlist_index[0]+1] = df_date["DELAY_ARR"][train_index] 
        
    return training_xi

In [10]:
training_xi = training_x_values(arrivalDataframe,arriving_trains_list,days_list)
training_xi

array([[ 1.000e+00, -1.500e+01, -9.000e+00, ...,  1.110e+02,  0.000e+00,
         0.000e+00],
       [ 1.000e+00,  2.069e+03,  1.580e+02, ...,  7.700e+01,  0.000e+00,
         0.000e+00],
       [ 1.000e+00, -1.000e+01,  1.020e+02, ..., -6.000e+00,  0.000e+00,
         0.000e+00],
       ...,
       [ 1.000e+00,  1.090e+02,  1.020e+02, ...,  2.540e+02,  5.300e+01,
        -2.400e+01],
       [ 1.000e+00,  1.400e+01,  1.890e+02, ...,  2.700e+02,  3.130e+02,
        -2.300e+01],
       [ 1.000e+00,  4.300e+01,  2.200e+02, ...,  1.590e+02,  1.550e+02,
         2.070e+02]])

Trein: heeft enkel 28 van de 30 dagen gereden! (gecancelt op andere dagen)

In [11]:
training_yi = df2["DELAY_DEP"].values
training_yi

array([ 2.800e+01,  9.100e+01,  6.500e+01,  2.400e+01,  8.200e+01,
        1.700e+01,  8.300e+01,  1.950e+02,  5.800e+01,  2.600e+01,
        3.000e+01,  2.900e+01,  2.600e+01,  7.100e+01,  4.500e+01,
        1.430e+02,  8.700e+01,  1.000e+01,  7.600e+01,  7.600e+01,
        7.580e+02,  6.700e+01,  0.000e+00,  4.200e+01,  7.100e+01,
        7.800e+01,  2.070e+02,  8.100e+01,  1.440e+02,  3.800e+01,
        2.830e+02,  2.400e+01,  9.000e+00,  2.400e+01,  7.760e+02,
        3.700e+01,  1.700e+01,  3.900e+01,  1.030e+02,  1.650e+02,
        4.000e+01,  4.500e+01,  6.000e+01,  1.600e+01,  2.400e+01,
        5.900e+01,  4.700e+01,  9.500e+01,  1.990e+02,  3.000e+00,
        1.090e+02,  9.000e+01, -5.000e+00,  1.032e+03,  3.490e+02,
        9.100e+01,  1.190e+02,  1.600e+01, -8.000e+00,  2.000e+00,
        1.340e+02,  6.500e+01,  4.600e+01,  2.360e+02,  1.050e+02,
        7.400e+01,  2.540e+02,  8.000e+00,  6.000e+00,  3.130e+02,
        1.070e+02,  2.300e+01,  1.290e+02,  3.900e+01,  1.280e

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(training_xi, training_yi, test_size=0.2, random_state=0)

Gecancelde treins weggefilterd --> meer bias omdat ik die waarden eerst op 0 had gezet?

In [80]:
regressor = LinearRegression()
linreg = regressor.fit(X_train, y_train)
y_pred = linreg.predict(X_test)
W = linreg.coef_[1:16]
print('linear model coeff (w):',W)
print('linear model intercept (b): {}'.format("%.3f" %linreg.intercept_), '\n')
r_sq = linreg.score(X_train, y_train)
print(f"coefficient of determination: {r_sq}")

print(arriving_trains_list)

linear model coeff (w): [-0.0257249   0.91840684 -0.02411156  0.01732823  0.01373751  0.04531939
 -0.00968226 -0.01683276 -0.00471466 -0.07424179 -0.03691383  0.11231019
  0.0312248   0.04049648  0.15189609]
linear model intercept (b): -12.742 

coefficient of determination: 0.6945185405820574
[2636 1815 1764 2887  736 2936 2986 4215 1964 3315 4336 2686 3115 3036
 1487]


In [87]:
from statsmodels.api import OLS
OLS(y_pred,X_test).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 5.979e+29
Date:                Fri, 25 Nov 2022   Prob (F-statistic):               0.00
Time:                        13:33:40   Log-Likelihood:                 1979.1
No. Observations:                  73   AIC:                            -3926.
Df Residuals:                      57   BIC:                            -3889.
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        -12.7423   1.04e-13  -1.22e+14      0.000     -12.742     -12.742
x1            -0.0257   2.43e-16  -1.06e+14      0.000      -0.026      -0.026
x2             0.9184   5.25e-16   1.75e+15      0.000       0.918       0.918
x3            -0.0241   4.93e-16  -4.89e+13      0.000      -0.024      -0.024
x4             0.0173   2.54e-16   6.81e+13      0.000       0.017       0.017
x5             0.0137    4.9e-16    2.8e+13      0.000       0.014       0.014
x6             0.0453    2.8e-16   1.62e+14      0.000       0.045       0.045
x7            -0.0097   5.09e-16   -1.9e+13      0.000      -0.010      -0.010
x8            -0.0168   5.63e-16  -2.99e+13      0.000      -0.017      -0.017
x9            -0.0047   3.77e-16  -1.25e+13      0.000      -0.005      -0.005
x10           -0.0742   3.98e-16  -1.86e+14      0.000      -0.074      -0.074
x11           -0.0369   5.06e-16  -7.29e+13      0.000      -0.037      -0.037
x12            0.1123   6.45e-16   1.74e+14      0.000       0.112       0.112
x13            0.0312   2.96e-16   1.05e+14      0.000       0.031       0.031
x14            0.0405   2.57e-16   1.58e+14      0.000       0.040       0.040
x15            0.1519   4.91e-16    3.1e+14      0.000       0.152       0.152
==============================================================================
Omnibus:                       18.935   Durbin-Watson:                   0.944
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               23.106
Skew:                          -1.262   Prob(JB):                     9.61e-06
Kurtosis:                       4.106   Cond. No.                     1.13e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.13e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [64]:
df_regression = arrivalDataframe
df_regression
df_regression = df_regression[df_regression["DATDEP"] == "01JUL2020"]
Last1train = arrivalDataframe[arrivalDataframe["TRAIN_NO"] == 3036]
Last1train = Last1train[Last1train["DATDEP"] == "31AUG2020"]
Last2train = arrivalDataframe[arrivalDataframe["TRAIN_NO"] == 1487]
Last2train = Last2train[Last2train["DATDEP"] == "15DEC2020"]
frames = [df_regression,Last1train,Last2train]
df_regression = pd.concat(frames)
df_regression["REGRESSION COEFFICIENT"] = W
cols = df_regression.columns.tolist()
cols = cols[2:6] + cols[9:10] + cols[-1:] + cols[13:16]
df_regression = df_regression[cols]
df_regression = df_regression.sort_values("PLANNED_TIME_ARR")
df_regression.style.hide_index()

C:\Users\tkaya\AppData\Local\Temp\ipykernel_15748\2790752155.py:15: FutureWarning: this method is deprecated in favour of `Styler.hide(axis='index')`
  df_regression.style.hide_index()


TRAIN_NO,RELATION,TRAIN_SERV,PTCAR_NO,PLANNED_TIME_ARR,REGRESSION COEFFICIENT,RELATION_DIRECTION,PTCAR_LG_NM_NL,LINE_NO_ARR
3036,IC 28,SNCB/NMBS,37,16:09:00,0.040496,IC 28: DE PANNE -> ANTWERPEN-CENTRAAL,ANTWERPEN-CENTRAAL,59/1
2636,IC 08,SNCB/NMBS,37,16:11:00,-0.025725,IC 08: HASSELT -> ANTWERPEN-CENTRAAL,ANTWERPEN-CENTRAAL,25
4215,IC 30,SNCB/NMBS,37,16:13:00,-0.016833,IC 30: TURNHOUT -> ANTWERPEN-CENTRAAL,ANTWERPEN-CENTRAAL,59/1
1764,L B1-1,SNCB/NMBS,37,16:20:00,-0.024112,L B1-1: NIVELLES -> ANTWERPEN-CENTRAAL,ANTWERPEN-CENTRAAL,27
736,IC 04-1,SNCB/NMBS,37,16:23:00,0.013738,IC 04-1: POPERINGE -> ANTWERPEN-CENTRAAL,ANTWERPEN-CENTRAAL,59/1
3315,IC 22,SNCB/NMBS,37,16:23:00,-0.074242,IC 22: BRUSSEL-ZUID -> ANTWERPEN-CENTRAAL,ANTWERPEN-CENTRAAL,27
2936,IC 09,SNCB/NMBS,37,16:30:00,0.045319,IC 09: LEUVEN -> ANTWERPEN-CENTRAAL,ANTWERPEN-CENTRAAL,27
2986,L A3,SNCB/NMBS,37,16:37:00,-0.009682,L A3: MOL -> ANTWERPEN-CENTRAAL,ANTWERPEN-CENTRAAL,59/1
2686,L A4,SNCB/NMBS,37,16:41:00,0.112310,L A4: LOKEREN -> ANTWERPEN-CENTRAAL,ANTWERPEN-CENTRAAL,59/1
4336,IC 10-1,SNCB/NMBS,37,16:51:00,-0.036914,IC 10-1: HAMONT -> ANTWERPEN-CENTRAAL,ANTWERPEN-CENTRAAL,59/1


In [71]:
now = datetime.datetime.now()
dt_string = now.strftime("%d-%m-%Y_%H-%M-%S")
csvName = "LinearRegression2years" + dt_string + ".csv"
df_regression.to_csv(csvName,index = False)

Vergelijking met arrival train:

In [32]:
df_aankomst = df2[df2["DATDEP"] == "01JUL2020"]
cols = df_aankomst.columns.tolist()
cols = cols[2:7] + cols[10:11] + cols[-1:] + cols[13:15]
df_aankomst = df_aankomst[cols]
df_aankomst
df_aankomst.style.hide_index()

C:\Users\tkaya\AppData\Local\Temp\ipykernel_15748\131865204.py:6: FutureWarning: this method is deprecated in favour of `Styler.hide(axis='index')`
  df_aankomst.style.hide_index()


TRAIN_NO,RELATION,TRAIN_SERV,PTCAR_NO,LINE_NO_DEP,PLANNED_TIME_DEP,REAL_DATE_DEP,RELATION_DIRECTION,PTCAR_LG_NM_NL
1838,IC 02,SNCB/NMBS,37,59/1,17:06:00,01JUL2020,IC 02: ANTWERPEN-CENTRAAL -> OOSTENDE,ANTWERPEN-CENTRAAL


Afleidingen:
1) Grote correlatie met dezelfde trein in de andere richting (de trein kan niet vertrekken voordat het aankomt)
2) Grote correlatie met de trein van 17h05 (1 minuut voor de beschoude trein) --> zelfde line NO
3) Negatieve correlaties met treinen die bijna 1 uur vroeger aankomen?


Notes:
1) Trein kan gecancelt zijn --> niet in database
2) Aankomende treinen kunnen ook gecancelt zijn --> niet in database
3) Een trein kan afhankelijk van weekend/werkdag een ander eindstations hebben:
vb: Trein 664 gaat in de week enkel van Gent tot Lokeren, maar in het weekend van Gent tot Antwerpen
4) Treinregeling wordt om de 6 maanden aangepast